In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler  # for normalization
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.svm import SVC
import torch

In [2]:
df = pd.read_csv('WhiteWineQuality.csv')
df.head()

,"fixed acidity;""volatile acidity"";""citric acid"";""residual sugar"";""chlorides"";""free sulfur dioxide"";""total sulfur dioxide"";""density"";""pH"";""sulphates"";""alcohol"";""quality"""
0,7;0.27;0.36;20.7;0.045;45;170;1.001;3;0.45;8.8;6
1,6.3;0.3;0.34;1.6;0.049;14;132;0.994;3.3;0.49;9...
2,8.1;0.28;0.4;6.9;0.05;30;97;0.9951;3.26;0.44;1...
3,7.2;0.23;0.32;8.5;0.058;47;186;0.9956;3.19;0.4...
4,7.2;0.23;0.32;8.5;0.058;47;186;0.9956;3.19;0.4...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 1 columns):
 #   Column                                                                                                                                                                   Non-Null Count  Dtype 
---  ------                                                                                                                                                                   --------------  ----- 
 0   fixed acidity;"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol";"quality"  4898 non-null   object
dtypes: object(1)
memory usage: 38.4+ KB


In [4]:
cols = list(df.columns)[0].split(';')
cols = [item.replace('"', '') for item in cols]
cols

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [5]:
vals = []
for i in range(len(df)):
    vals.append(list(df.values[i])[0].split(';'))
vals[0]

['7',
 '0.27',
 '0.36',
 '20.7',
 '0.045',
 '45',
 '170',
 '1.001',
 '3',
 '0.45',
 '8.8',
 '6']

In [6]:
data = pd.DataFrame(vals, columns = cols)
data = data.astype(float)
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6.0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6.0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6.0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6.0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6.0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   float64
dtypes: float64(12)
memory usage: 459.3 KB


In [26]:
# initially implementing binary classification
data = data[(data['quality'] == 6) | (data['quality'] == 5)]
x = data.drop('quality', axis = 1)
y = pd.to_numeric(data['quality'])
xnorm = MinMaxScaler().fit_transform(x)
xnorm[:5]

array([[0.30769231, 0.21468927, 0.21686747, 0.30828221, 0.10682493,
        0.33333333, 0.48059701, 0.26622875, 0.25688073, 0.26506024,
        0.13333333],
       [0.24038462, 0.24858757, 0.20481928, 0.01533742, 0.11869436,
        0.09302326, 0.36716418, 0.13098918, 0.53211009, 0.31325301,
        0.25      ],
       [0.41346154, 0.2259887 , 0.24096386, 0.09662577, 0.12166172,
        0.21705426, 0.26268657, 0.15224111, 0.49541284, 0.25301205,
        0.35      ],
       [0.32692308, 0.16949153, 0.19277108, 0.12116564, 0.14540059,
        0.34883721, 0.52835821, 0.16190108, 0.43119266, 0.20481928,
        0.31666667],
       [0.32692308, 0.16949153, 0.19277108, 0.12116564, 0.14540059,
        0.34883721, 0.52835821, 0.16190108, 0.43119266, 0.20481928,
        0.31666667]])

In [27]:
y = np.where(y == 5, -1, 1)
xtr, xtest, ytr, ytest = train_test_split(xnorm, y, test_size = 0.15, random_state = 42)
xtr.shape, ytr.shape, xtest.shape, ytest.shape

((3106, 11), (3106,), (549, 11), (549,))

In [29]:
xtr = torch.tensor(xtr, dtype = torch.float32, requires_grad = False)
ytr = torch.tensor(ytr, dtype = torch.float32, requires_grad = False)
xtest = torch.tensor(xtest, dtype = torch.float32, requires_grad = False)
xtr.shape, ytr.shape

(torch.Size([3106, 11]), torch.Size([3106]))

In [40]:
# hinge loss function - L(y,f(x))=max(0,1−y⋅f(x))
# the J(w,b) = 1/2 * det(w) **2 + c * sum(max(0, 1 - yi * (wT @ xi + b)))
# here the output of yi * (wT @ xi + b) is an [1] vector, moreover the shapes of yi is [1], wT is [1, m] and xi is [m, 1] where m is the no of featueres
# and shape of b is [1]. For easier implementation xi @ w, which will give the exact same result because shape of xi in each iteration will be [1, m] and
# shape of w is [m, 1].
# dJ/dw = w if max(0, 1 - yi * (wT @ xi + b)) == 0
#         w - c * sum(yi * xi) otherwise
# dJ/db = -c * sum(yi)

In [ ]:
# implementing using stochastic gradient descent - where the cost is computed separately for each variable
# hyper parameters
c = 10
learning_rate = 1.0
num_epochs = 100
w = torch.randn(xtr.shape[1], requires_grad = False)
b = torch.ones(1, requires_grad = False)
print(w.shape, b.shape)

for i in range (num_epochs):
    total_loss = 0.0
    for j in range(len(xtr)):
        xi = xtr[j]
        yi = ytr[j]
        # Compute the margin
        margin = yi * (torch.dot(xi, w) + b)
        loss = torch.max(torch.tensor(0.0), 1 - margin)
        total_loss += loss.item()
        
        if margin < 1:
            w = w - learning_rate * (w - c * ytr[i] * xtr[i])
            b = b - learning_rate * (-c * ytr[i])
        else:
            w = w - learning_rate * w
    print(f'loss for iteration {i} is {total_loss}')
w = w.numpy()
b = b.item()

torch.Size([11]) torch.Size([1])
loss for iteration 0 is 17786110.78923416
loss for iteration 1 is 17779474.77340126
loss for iteration 2 is 17803303.99964142
loss for iteration 3 is 17804281.82913208
loss for iteration 4 is 7429901.202720642
loss for iteration 5 is 7434477.276902676
loss for iteration 6 is 17936929.331993103
loss for iteration 7 is 53469160.01171875
loss for iteration 8 is 53468133.07421875
loss for iteration 9 is 17934030.725212097
loss for iteration 10 is 17937677.046020508
loss for iteration 11 is 53468167.04296875
loss for iteration 12 is 53468679.029296875
loss for iteration 13 is 17936154.674568176
loss for iteration 14 is 7345822.751958847
loss for iteration 15 is 7350574.397658825
loss for iteration 16 is 18013479.720565796
loss for iteration 17 is 18011461.065490723
loss for iteration 18 is 7347687.117868423
loss for iteration 19 is 22253604.961914062
loss for iteration 20 is 37161551.6484375
loss for iteration 21 is 52070291.65625
loss for iteration 22 is 66

In [49]:
def predict(x, w, b):
    decision_values = (x @ w) + b
    return np.where(decision_values >= 0, 1, -1)

pred = predict(xtest, w, b)
print(f'recall score {recall_score(ytest, pred)}')
print(f'precision score {precision_score(ytest, pred)}')

recall score 1.0
precision score 0.5701275045537341


/tmp/ipykernel_46985/2332971683.py:2: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  decision_values = (x @ w) + b
